In [40]:
from braket.tracking import Tracker
t = Tracker().start()

In [41]:
from qiskit_braket_provider import BraketLocalBackend
local_simulator = BraketLocalBackend()

In [42]:
local_dm_simulator = BraketLocalBackend(name='braket_dm')

In [43]:
from qiskit_braket_provider import AWSBraketProvider

provider = AWSBraketProvider()

provider.backends(statuses=["ONLINE"], types=["SIMULATOR"])

[BraketBackend[SV1], BraketBackend[TN1], BraketBackend[dm1]]

In [44]:
aws_tn1 = provider.get_backend("TN1")

In [45]:
provider.backends(statuses=["ONLINE"], types=["QPU"])

[BraketBackend[Forte 1], BraketBackend[Harmony], BraketBackend[Lucy]]

In [46]:
# qpu_backend = provider.get_backend("Aspen-M-3")
#qpu_backend = provider.get_backend("Lucy")
#qpu_backend = provider.get_backend("Aria 1")
#qpu_backend = provider.get_backend("Forte 1")
qpu_backend = provider.get_backend("Lucy")

print(qpu_backend)

BraketBackend[Lucy]


In [47]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import numpy as np

In [48]:
#the expression considered is of the form f(x,y) = ax^2 + bxy + cy^2 + dx + ey + f
#where x is binary variable and y is a non- binary variable

a = 1
b = -2
c = 2
d = -1
e = 1
f = 0

In [49]:
#Defining number of qubits for each variable
n_qubits_x = 2  # For binary variable x
n_qubits_y = 3  # For non-binary variable y

#Creating the QAOA quantum circuit
p = 1  # Number of adiabatic steps
qaoa_circuit = QuantumCircuit(n_qubits_x + n_qubits_y, n_qubits_x + n_qubits_y)

# Applying Hadamard gate to all qubits
qaoa_circuit.h(range(n_qubits_x + n_qubits_y))

# Applying X and Z rotations according to the cost function
qaoa_circuit.rx(2 * a * p, range(n_qubits_x))
qaoa_circuit.rz(2 * b * p, range(n_qubits_x, n_qubits_x + n_qubits_y))
qaoa_circuit.rx(2 * c * p, range(n_qubits_x + n_qubits_y))
qaoa_circuit.rz(2 * d * p, range(n_qubits_x))
qaoa_circuit.rz(2 * e * p, range(n_qubits_x, n_qubits_x + n_qubits_y))

# Measuring qubits for both x and y
qaoa_circuit.measure(range(n_qubits_x + n_qubits_y), range(n_qubits_x + n_qubits_y))

In [57]:
# Simulating the quantum circuit
backend = qpu_backend
shots = 1000
t_qaoa = transpile(qaoa_circuit, backend)
#qobj = assemble(t_qaoa, shots=shots)
#result = backend.run(qobj).result()
result = backend.run(t_qaoa, shots=shots).result()

#counts = result.get_counts(qaoa_circuit)
counts = result.get_counts
print("Measurement results:", counts)

Measurement results: <bound method Result.get_counts of Result(backend_name='BraketBackend[Lucy]', backend_version='2', qobj_id='0', job_id='arn:aws:braket:eu-west-2:891377090278:quantum-task/f9f71c6a-5bd2-4f08-805a-4f1743d23223', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'00000': 11, '10000': 27, '01000': 27, '11000': 31, '00100': 32, '10100': 35, '01100': 38, '11100': 68, '00010': 11, '10010': 15, '01010': 23, '11010': 44, '00110': 19, '10110': 40, '01110': 35, '11110': 43, '00001': 12, '10001': 20, '01001': 22, '11001': 44, '00101': 28, '10101': 48, '01101': 48, '11101': 53, '00011': 11, '10011': 17, '01011': 28, '11011': 41, '00111': 23, '10111': 22, '01111': 36, '11111': 48}, memory=['00000', '00000', '00000', '00000', '00000', '00000', '00000', '00000', '00000', '00000', '00000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '10000', '1000

In [56]:
most_probable_solution_x = max(counts, key=counts)
x_value = int(most_probable_solution_x[:n_qubits_x], 2)

most_probable_solution_y = max(counts, key=counts)[n_qubits_x:]
y_value = int(most_probable_solution_y, 2)

print("Most probable solution for x:", x_value)
print("Most probable solution for y:", y_value)

TypeError: 'method' object is not iterable

In [51]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.2f} USD")

Quantum Task Summary
{'arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy': {'shots': 1000, 'tasks': {'COMPLETED': 1}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 0.65 USD
